In [42]:
%%writefile awkward.py

import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class awkwardClassifier(nn.Module):

    def __init__(self, 
                 ntoken, # 단어수
                 ntoken2,
                 ninp,   # 임베딩 차원
                 n_classes,
                 nhead, 
                 nhid, 
                 nlayers,
                 use_batch_norm=False,
                 dropout=0.5):

        super(awkwardClassifier, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        
        self.ninp = ninp
        self.encoder = nn.Embedding(ntoken, ninp, padding_idx=1)
        self.encoder2 = nn.Embedding(ntoken2, ninp, padding_idx=1)
        
        ## 첫번째 문장
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        #self.decoder = nn.Linear(ninp, ntoken)

        ## 두번째 문장
        self.pos_encoder2 = PositionalEncoding(ninp, dropout)
        encoder_layers2 = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder2 = TransformerEncoder(encoder_layers2, nlayers)
        #self.decoder2 = nn.Linear(ninp, ntoken)
        
        ## 문장간 유사도
        #self.con_li = nn.Linear(ninp, n_classes)
        self.con_li = nn.Linear(64, n_classes) # 시퀀스 길이 x 클래스 갯수
        self.activation = nn.LogSoftmax(dim=-1)
        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.encoder2.weight.data.uniform_(-initrange, initrange)
        self.con_li.weight.data.uniform_(-initrange, initrange)

        #self.decoder.bias.data.zero_()
        #self.decoder.weight.data.uniform_(-initrange, initrange)
        
        #self.decoder2.bias.data.zero_()
        #self.decoder2.weight.data.uniform_(-initrange, initrange)
        #self.result.weight.data.uniform_(-initrange, initrange)


    def forward(self, src, src2):
        
        ## 첫번째 문장
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        #output = self.decoder(output)
        
        ## 두번째 문장
        if self.src_mask is None or self.src_mask.size(0) != len(src2):
            device = src2.device
            mask = self._generate_square_subsequent_mask(len(src2)).to(device)
            self.src_mask = mask

        src2 = self.encoder2(src2) * math.sqrt(self.ninp)
        src2 = self.pos_encoder2(src2)
        output2 = self.transformer_encoder2(src2, self.src_mask)
        # output2 = self.decoder2(output2)

        #final_outs = torch.cat([output, output2], dim = 2)
        ## <1차 성공>
        #  final_outs = torch.add(output, output2)
        #  output = self.con_li(final_outs)

        # y = self.activation(output[:, -1])
        ## </1차 성공>
    
        ## <2차 시도>
        #final_outs = torch.exp(-torch.sum(torch.abs(output - output2), dim=2)) # 성공 (7, 64) -> 모두 0 아니면 1로
        final_outs = -torch.sum(torch.abs(output - output2), dim=2) # 성공 (7, 64) -> torch.exp적용시 정확도가 떨어지는 현상 보여 제외함.
        output = self.con_li(final_outs)
        y = self.activation(output)
        
        return y
        #return torch.argmax(y, dim=1)

    
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)
    

Overwriting awkward.py


In [ ]:
# gen_x -> (9, 256)

<h1>Backup</h1>

200614_v1 : 한문장, 라벨 분류

In [ ]:
%%writefile awkward.py

import torch
import torch.nn as nn

class awkwardClassifier(nn.Module):
    
    def __init__(
        self,
        input_size,
        word_vec_size,
        n_classes,
        use_batch_norm=False,
        dropout_p=.5
    ):
        
        self.input_size = input_size  # vocabulary size
        self.word_vec_size = word_vec_size
        self.n_classes = n_classes
        self.use_batch_norm = use_batch_norm
        self.dropout_p = dropout_p

        super().__init__()
        
        self.emb = nn.Embedding(input_size, word_vec_size)
        self.generator  = nn.Linear(word_vec_size, n_classes)
        self.activation = nn.LogSoftmax(dim=-1)

    def forward(self, x):
        
        # |x| = (batch_size, length, word_vec_size)
        x = self.emb(x)
        
        #|gen_x| = (batch_size, length, n_classes)
        #print(gen_x.shape) [9, 64, 3]
        gen_x = self.generator(x)

        # |y| = (batch_size, n_classes)
        y = self.activation(gen_x[:, -1])
        
        return y

<h1>Backup</h1>

200614_v2: 두문장, 라벨 분류

In [ ]:
%%writefile awkward.py

import torch
import torch.nn as nn

class awkwardClassifier(nn.Module):
    
    def __init__(
        self,
        input_size,
        word_vec_size,
        n_classes,
        use_batch_norm=False,
        dropout_p=.5
    ):
        
        self.input_size = input_size  # vocabulary size
        self.word_vec_size = word_vec_size
        self.n_classes = n_classes
        self.use_batch_norm = use_batch_norm
        self.dropout_p = dropout_p

        super().__init__()
        
        self.emb = nn.Embedding(input_size, word_vec_size)
        
        self.generator  = nn.Linear(word_vec_size, n_classes)
        self.generator_x2  = nn.Linear(word_vec_size, n_classes)
        
        self.concat  = nn.Linear(n_classes * 2, n_classes)
        
        self.activation = nn.LogSoftmax(dim=-1)

    def forward(self, x, x2):
        
        # |x| = (batch_size, length, word_vec_size)
        x = self.emb(x)
        x2 = self.emb(x2)
        
        #|gen_x| = (batch_size, length, n_classes)
        #print(gen_x.shape) [9, 64, 3]
        gen_x = self.generator(x)
        gen_x2 = self.generator_x2(x)
        
        gen_outs = torch.cat((gen_x, gen_x2), dim=-1)
        print("gen_outs", gen_outs.shape)
        
        y_hat = self.concat(gen_outs)
        
        print("y_hat", y_hat.shape)
        # |y| = (batch_size, n_classes)
        y = self.activation(y_hat[:, -1])
        
        return y

<h1>Backup</h1>

200614_v3: 두문장, 라벨 분류

In [ ]:
%%writefile awkward.py

import torch
import torch.nn as nn

class awkwardClassifier(nn.Module):
    
    def __init__(
        self,
        input_size,
        word_vec_size,
        n_classes,
        use_batch_norm=False,
        dropout_p=.5
    ):
        
        self.input_size = input_size  # vocabulary size
        self.word_vec_size = word_vec_size
        self.n_classes = n_classes
        self.use_batch_norm = use_batch_norm
        self.dropout_p = dropout_p

        super().__init__()
        
        self.emb = nn.Embedding(input_size, word_vec_size)
        
        self.generator  = nn.Linear(word_vec_size, n_classes)
        self.generator_x2  = nn.Linear(word_vec_size, n_classes)
        
        self.concat  = nn.Linear(1, n_classes)
        
        self.activation = nn.LogSoftmax(dim=-1)

    def forward(self, x, x2):
        
        # |x| = (batch_size, length, word_vec_size)
        x = self.emb(x)
        x2 = self.emb(x2)
        
        #|gen_x| = (batch_size, length, n_classes)
        #print(gen_x.shape) [9, 64, 3]
        gen_x = self.generator(x)
        gen_x2 = self.generator_x2(x)
        
        gen_outs = torch.exp(-torch.sum(torch.abs(gen_x - gen_x2), dim=2, keepdim=True))
        
        y_hat = self.concat(gen_outs)
        
        # |y| = (batch_size, n_classes)
        y = self.activation(y_hat[:, -1])
        
        return y

<h1>Backup</h1>

200614_v4: 두문장, 라벨 분류

In [ ]:
%%writefile awkward.py

import torch
import torch.nn as nn

class awkwardClassifier(nn.Module):
    
    def __init__(
        self,
        input_size,
        word_vec_size,
        n_classes,
        use_batch_norm=False,
        dropout_p=.5
    ):
        
        self.input_size = input_size  # vocabulary size
        self.word_vec_size = word_vec_size
        self.n_classes = n_classes
        self.use_batch_norm = use_batch_norm
        self.dropout_p = dropout_p

        super().__init__()
        
        self.emb = nn.Embedding(input_size, word_vec_size)
        
        self.generator  = nn.Linear(word_vec_size, n_classes)
        self.generator_x2  = nn.Linear(word_vec_size, n_classes)
        
        self.concat  = nn.Linear(1, n_classes)
        
        self.activation = nn.LogSoftmax(dim=-1)

    def forward(self, x, x2):
        
        # |x| = (batch_size, length, word_vec_size)
        x = self.emb(x)
        x2 = self.emb(x2)
        
        #|gen_x| = (batch_size, length, n_classes)
        #print(gen_x.shape) [9, 64, 3]
        gen_x = self.generator(x)
        gen_x2 = self.generator_x2(x)
        
        gen_outs = torch.exp(-torch.sum(torch.abs(gen_x - gen_x2), dim=2, keepdim=True))
        
        y_hat = self.concat(gen_outs)
        
        # |y| = (batch_size, n_classes)
        y = self.activation(y_hat[:, -1])
        
        return y

<h1>Backup</h1>

200614_v5: 트랜스포머, 한문장, 라벨 분류

In [ ]:
%%writefile awkward.py

import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class awkwardClassifier(nn.Module):

    def __init__(self, 
                 ntoken, 
                 ninp,  
                 n_classes,
                 nhead, 
                 nhid, 
                 nlayers,
                 use_batch_norm=False,
                 dropout=0.5):

        super(awkwardClassifier, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        
        ## 첫번째 문장
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)
        
        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)
        
    def forward(self, src, src2):
        
        ## 첫번째 문장
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        
        return output[:,-1]
    
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

<h1>Backup</h1>

200614_v6: 10건 테스트 정상 학습, 트랜스포머, 두문장, 라벨 분류

In [ ]:
%%writefile awkward.py

import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class awkwardClassifier(nn.Module):

    def __init__(self, 
                 ntoken, # 단어수
                 ninp,   # 임베딩 차원
                 n_classes,
                 nhead, 
                 nhid, 
                 nlayers,
                 use_batch_norm=False,
                 dropout=0.5):

        super(awkwardClassifier, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        
        ## 첫번째 문장        
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)
        
        ## 두번째 문장        
        self.pos_encoder2 = PositionalEncoding(ninp, dropout)
        encoder_layers2 = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder2 = TransformerEncoder(encoder_layers2, nlayers)
        self.encoder2 = nn.Embedding(ntoken, ninp)
        self.ninp2 = ninp
        self.decoder2 = nn.Linear(ninp, ntoken)
        
        ## 문장간 유사도
        self.con_li = nn.Linear(ntoken, n_classes)
        self.activation = nn.LogSoftmax(dim=-1)
        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)
        
#         self.encoder2.weight.data.uniform_(-initrange, initrange)
#         self.decoder2.bias.data.zero_()
#         self.decoder2.weight.data.uniform_(-initrange, initrange)
        
#         self.result.weight.data.uniform_(-initrange, initrange)


    def forward(self, src, src2):
        
        ## 첫번째 문장
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        
        ## 두번째 문장
        if self.src_mask is None or self.src_mask.size(0) != len(src2):
            device = src2.device
            mask = self._generate_square_subsequent_mask(len(src2)).to(device)
            self.src_mask = mask

        src2 = self.encoder2(src2) * math.sqrt(self.ninp2)
        src2 = self.pos_encoder2(src2)
        output2 = self.transformer_encoder2(src2, self.src_mask)
        output2 = self.decoder2(output2)
        
        ## 두 문장간 유사도
        #final_outs = torch.exp(-torch.sum(torch.abs(output - output2), dim=2, keepdim=True))
        #final_outs = result(torch.exp(-torch.sum(torch.abs(output - output2), dim=2, keepdim=True)))
        
        print("output",output.shape)
        print("output2",output2.shape)
        
        final_outs = torch.add(output, output2)
        output = self.con_li(final_outs)

        y = self.activation(output[:, -1])
        return y
    
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)
    

In [2]:
from torchtext import data
from torchtext import data
from konlpy.tag import Okt 
import torch

okt = Okt()

valid_ratio=0.1
use_eos = True
batch_size = 3
device = torch.cuda.is_available()
shuffle = True
max_vocab = 9999999
min_freq = 1

# Define field of the input file.
# The input file consists of two fields.
text1 = data.Field(tokenize=okt.morphs,
                   use_vocab = True,
                   batch_first = True,
                   include_lengths = False,
                   eos_token='<EOS>' if use_eos else None,
                   pad_first=True)

text2 = data.Field(tokenize=okt.morphs,
                   use_vocab = True,
                   batch_first = True,
                   include_lengths = False,
                   eos_token='<EOS>' if use_eos else None,
                   pad_first=True)

# sequential : If False,no tokenization is applied
label = data.Field(sequential = False,
                   use_vocab = True,
                   init_token=None,
                   unk_token = None)

# Those defined two columns will be delimited by TAB.
# Thus, we use TabularDataset to load two columns in the input file.
# We would have two separate input file: train_fn, valid_fn
# Files consist of two columns: label field and text field.
train, valid = data.TabularDataset(
    path="/home/work/data/awkward_v2/data/albert_pretrain/sen_sen_label_v1_test.csv",
    format='tsv', 
    fields=[
        ('text1', text1),
        ('text2', text2),
        ('label', label),
    ],
).split(split_ratio=(1 - valid_ratio))


In [3]:
# Those loaded dataset would be feeded into each iterator:
# train iterator and valid iterator.
# We sort input sentences by length, to group similar lengths.
train_loader, valid_loader = data.BucketIterator.splits(
    (train, valid),
    batch_size=batch_size,
    device='cuda:%d' % device if device >= 0 else 'cpu',
    shuffle=shuffle,
    sort_key=lambda x: len(x.text1),
    sort_within_batch=True,
)

In [4]:
# At last, we make a vocabulary for label and text field.
# It is making mapping table between words and indice.
label.build_vocab(train)
text1.build_vocab(train, max_size=max_vocab, min_freq=min_freq)
text2.build_vocab(train, max_size=max_vocab, min_freq=min_freq)

In [ ]:
# 길이가 다르다.
for x, y in train_loader:
    print(x[0].shape)
    print(x[0])
    
    #print(x[1].shape)
    #print(x[1])
    
    #print(x[2].shape)
    print("구분")
    #padding_value (float, optional) – values for padded elements. // total_length
    
    packed = pack_padded_sequence(x[0], total_length = [32], batch_first=True, enforce_sorted=False)
    print(packed)
    
    break

In [ ]:
## Step 4: Pad instances with 0s till max length sequence ##
##--------------------------------------------------------##
# get the length of each seq in your batch
seq_lengths = LongTensor(list(map(len, vectorized_seqs)))
# seq_lengths => [ 8, 4,  6]
# batch_sum_seq_len: 8 + 4 + 6 = 18
# max_seq_len: 8

seq_tensor = Variable(torch.zeros((len(vectorized_seqs), seq_lengths.max()))).long()
# seq_tensor => [[0 0 0 0 0 0 0 0]
#                [0 0 0 0 0 0 0 0]
#                [0 0 0 0 0 0 0 0]]

for idx, (seq, seqlen) in enumerate(zip(vectorized_seqs, seq_lengths)):
    seq_tensor[idx, :seqlen] = LongTensor(seq)
# seq_tensor => [[ 6  9  8  4  1 11 12 10]          # long_str
#                [12  5  8 14  0  0  0  0]          # tiny
#                [ 7  3  2  5 13  7  0  0]]         # medium
# seq_tensor.shape : (batch_size X max_seq_len) = (3 X 8)

In [ ]:
max_length = 32
batch_size = 3

In [ ]:
vectorized_seqs = [[6, 9, 8, 4, 1, 11, 12, 10],
                   [12, 5, 8, 14],
                   [7, 3, 2, 5, 13, 7]]

In [5]:
from torch import LongTensor
from torch.autograd import Variable

max_length= 32

# 길이가 다르다.
for x, y in train_loader:

    vectorized_seqs = x[0]
    #print(x[1])
    #print(x[2])
    

    #seq_lengths = LongTensor(list(map(len, vectorized_seqs)))
    #seq_tensor = Variable(torch.zeros(batch_size, max_length)).long()
    
    break

In [8]:
vectorized_seqs

tensor([[ 8, 23, 84, 43,  5, 92,  7, 44,  6, 16,  4, 71, 94, 50, 48,  3, 30, 73,
         34, 10, 67,  9, 18, 19, 95, 80, 22, 35, 45, 89, 76,  9, 15,  4, 12, 20,
         88, 77, 97, 42, 78, 46, 82,  5, 31, 98, 56, 47, 61,  9,  6, 58, 99, 15,
          4, 12, 20, 90, 19, 96, 52, 79, 57,  3,  2],
        [ 8, 23, 84, 43,  5, 92,  7, 44,  6, 16,  4, 71, 94, 50, 48,  3, 30, 73,
         34, 10, 67,  9, 18, 19, 95, 80, 22, 35, 45, 89, 76,  9, 15,  4, 12, 20,
         88, 77, 97, 42, 78, 46, 82,  5, 31, 98, 56, 47, 61,  9,  6, 58, 99, 15,
          4, 12, 20, 90, 19, 96, 52, 79, 57,  3,  2],
        [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         70, 13, 60, 17, 87, 13, 16,  4, 28, 29,  7, 14, 21, 49, 18, 54,  8,  6,
         72,  6, 81, 25, 11, 27, 17, 59, 51, 33, 91, 66, 65, 41, 36, 85,  7, 14,
         21, 63, 74,  5, 32, 75, 69, 24, 93,  3,  2]], device='cuda:1')

In [6]:
vectorized_seqs = p

In [9]:
seq_lengths = LongTensor(list(map(len, vectorized_seqs)))

In [11]:
seq_tensor = Variable(torch.zeros(len(seq_lengths), max_length)).long()

In [13]:
seq_lengths

tensor([65, 65, 65])

In [15]:
for idx, (seq, seqlen) in enumerate(zip(vectorized_seqs, seq_lengths)):
    
    #print(seq)
    #print(seqlen)
    
    seq_tensor[idx, :seqlen]

    print(seq_tensor)


tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
vectorized_seqs

In [ ]:
seq_tensor = Variable(torch.zeros(batch_size, max_length)).long()

In [ ]:
for idx, (seq, seqlen) in enumerate(zip(vectorized_seqs, seq_lengths)):

###### seq 길이 통일해주는 함수

In [17]:
import torch

In [7]:
a = torch.tensor(
        [[0, 0, 0, 0, 5, 0, 6, 0, 4, 0, 3, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0,
         0, 0, 4, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 2],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 3, 0, 0, 6, 0, 0, 0, 0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 6, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 2],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 3, 0, 0, 6, 0, 0, 0, 0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 6, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 2],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 2],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 2],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]])

In [8]:
b = torch.tensor(
        [[0, 0, 0, 0, 5, 0, 6, 0, 4, 0, 3, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0,
         0, 0, 4, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 2],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 3, 0, 0, 6, 0, 0, 0, 0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 6, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 2],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 3, 0, 0, 6, 0, 0, 0, 0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 6, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 2],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 2],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 2],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]])

In [62]:
a.size(0)

7

In [35]:
# input : tensor(batch x maxinum_length) / seq_length
# 1. 길때, 작을때 분기
# 1-1
# 1-2

In [63]:
maxinum_length = 84
pad = 1
batchsize = 7 

In [80]:
a.shape

torch.Size([7, 64])

In [82]:
if a.size(1) >= maxinum_length:
    a = a[:, :engine.config.max_length]
    #x2 = x2[:, :engine.config.max_length]

else:
    pad_num = maxinum_length - a.size(1)
    pad = torch.ones(batchsize, pad_num)
    output = torch.cat([pad.long(), a], dim=-1)

In [ ]:
gpu

In [118]:
def pad_to_maxseq_to_batch(batch, max_length, device=-1):
    
    if batch.size(1) >= max_length:
        batch = batch[:, :max_length]
        
    else:
        pad_num = max_length - batch.size(1)
        pad = torch.ones(batch.size(0), pad_num, device=device) # pad 값이 vocab index 1이라는 가정.
        batch = torch.cat([pad.long(), batch], dim=-1)

    return batch

In [119]:
device = torch.device("cuda")
pad_to_maxseq_to_batch(a, 128, device)

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 5, 0, 6, 0,
         4, 0, 3, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 3, 0,
         0, 0, 0, 0, 0, 0, 0, 2],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 6, 0, 0, 0,
         0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0,
         0, 5, 0, 0, 0, 0, 0, 2],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [84]:
output.shape

torch.Size([7, 84])

In [66]:
pad

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.]])

In [69]:
a.shape

torch.Size([7, 64])

In [70]:
pad.shape

torch.Size([7, 20])

In [77]:
torch.cat([pad.long(), a], dim=-1)

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         5, 0, 6, 0, 4, 0, 3, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 4, 0,
         0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 2],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0,
         6, 0, 0, 0, 0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         6, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 2],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0,
         6, 0, 0, 0, 0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         6, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 2],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [5]:
import torch

In [9]:
torch.minus(a - b)

AttributeError: module 'torch' has no attribute 'minus'